### 損失函數

> 機器學習中所有的算法都需要最⼤化或最⼩化⼀個函數，這個函數被稱為「⽬標函數」。其中，我們⼀般把最⼩化的⼀類函數，稱為「損失函數」。它能根據預測結果，衡量出模型預測能⼒的好壞。
> 損失函數⼤致可分為：分類問題的損失函數和回歸問題的損失函數

### 損失函數為什麼是最⼩化

> 期望：希望模型預測出來的東⻄可以跟實際的值⼀樣

> 預測出來的東⻄基本上跟實際值都會有落差

> 損失函數中的損失就是「實際值和預測值的落差」

- 在回歸問題稱為「殘差(residual)」
- 在分類問題稱為「錯誤率(errorrate)」


### 均⽅誤差(mean_squared_error)：
> 就是最⼩平⽅法(Least Square) 的⽬標函數 -- 預測值與實際值的差距之平均值。還有其他變形的函數, 如 mean_absolute_error、mean_absolute_percentage_error、mean_squared_logarithmic_error。

##### Keras 上的調⽤⽅式：

from keras import losses

model.compile(loss= ‘mean_squared_error‘, optimizer='sgd’)

#其中，包含 y_true， y_pred 的傳遞，函數是表達如下：

keras.losses.mean_squared_error(y_true, y_pred)

### 損失函數的分類介紹 - Cross Entropy
> 當預測值與實際值愈相近，損失函數就愈⼩，反之差距很⼤，就會更影響損失函數的值,要⽤ Cross Entropy 取代MSE，因為，在梯度下時，Cross Entropy 計算速度較快，

##### 使⽤時機：
- 整數⽬標：Sparse categorical_crossentropy
- 分類⽬標：categorical_crossentropy
- ⼆分類⽬標：binary_crossentropy。

##### Keras 上的調⽤⽅式：
from keras import losses

model.compile(loss= ‘categorical_crossentropy ‘, optimizer='sgd’)

#其中, 包含y_true， y_pred的傳遞, 函數是表達如下：

keras.losses.categorical_crossentropy(y_true, y_pred)

### 損失函數的分類介紹: Hinge Error (hinge)
> 是⼀種單邊誤差，不考慮負值,同樣也有多種變形，squared_hinge、categorical_hinge

##### 使⽤時機：
- 適⽤於『⽀援向量機』(SVM)的最⼤間隔分類法(maximum-margin classification)

##### Keras 上的調⽤⽅式：
from keras import losses

model.compile(loss= ‘hinge‘, optimizer='sgd’)

#其中，包含 y_true，y_pred 的傳遞, 函數是表達如下:

keras.losses.hinge(y_true, y_pred) 

### 特別的案例: ⾃定義損失函數
> 根據問題的實際情況，定制合理的損失函數

> 舉例：預測果汁⽇銷量問題，如果預測銷量⼤於實際銷量則會損失成本；如果預測銷量⼩於實際銷量則會損失利潤。

> 考慮重點：製造⼀盒果汁的成本和銷售⼀盒果汁的利潤不是等價的,需要使⽤符合該問題的⾃定義損失函數⾃定義損失函數為：

- 損失函數表⽰若預測結果 y ⼩於標準答案 y_，損失函數為利潤乘以預測結果 y 與標準答案之差
- 若預測結果 y ⼤於標準答案 y_，損失函數為成本乘以預測結果 y 與標準答案之差⽤

> Tensorflow 函數表⽰為：
> loss = tf.reduce_sum(tf.where(tf.greater(y, y_), COST*(y-y_), PROFIT*(y_-y)))

## 延伸閱讀

[官方文件](https://keras.io/losses/)

# 課程目標

利用Keras 的 CIFAR10 dataset 來驗證損失函數對於模型精確度的影響

# 範例重點: 
    
    模型架構的層數
    
    使用何種LOSS function

In [2]:
import keras
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)
#np.random.seed(10)的作用：使得隨機數據可預測

Using TensorFlow backend.


In [3]:
#取得Keras CIFAR10 Dataset, 並分成Training 與 Test set
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape)

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化, 並設定 data array 為浮點數
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE, 並查看維度資訊
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
# 宣告採用序列模型
model= Sequential()
#卷積層1與池化層1
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(Dropout(rate=0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#卷積層2與池化層2
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#建立神經網路(平坦層、隱藏層、輸出層)
model.add(Flatten())
model.add(Dropout(0.25))
#建立全網路連接層
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
#建立輸出層
model.add(Dense(10, activation='softmax'))
#檢查model 的STACK
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [8]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [ ]:
#模型編譯
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)         

#[validation_split = 0.2] validation_split：在0和1之間浮動。用作驗證數據的訓練數據的分數。
#該模型將訓練數據的這一部分分開，不會對其進行訓練，並將在每個時期結束時評估該數據的損失和任何模型指標。
#[batch_size]：整數或None。每個梯度更新的樣本數。指定，batch_size為128

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 136s 4ms/sample - loss: 1.5553 - accuracy: 0.4392 - val_loss: 1.3327 - val_accuracy: 0.5638
Epoch 2/12
37500/37500 [==============================] - 138s 4ms/sample - loss: 1.1785 - accuracy: 0.5836 - val_loss: 1.1468 - val_accuracy: 0.6202
Epoch 3/12
37500/37500 [==============================] - 135s 4ms/sample - loss: 1.0392 - accuracy: 0.6343 - val_loss: 1.0923 - val_accuracy: 0.6459
Epoch 4/12
14464/37500 [==========>...................] - ETA: 1:33 - loss: 0.9272 - accuracy: 0.6726

#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

In [ ]:
# 宣告採用序列模型
model= Sequential()
#卷積層1與池化層1
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(Dropout(rate=0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#卷積層2與池化層2
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#建立神經網路(平坦層、隱藏層、輸出層)
model.add(Flatten())
model.add(Dropout(0.25))
#建立全網路連接層
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
#建立輸出層
model.add(Dense(10, activation='softmax'))
#檢查model 的STACK
model.compile(loss='MSE', optimizer='Adam', metrics=['accuracy'])
train_history_MSE=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)  

In [ ]:
# 宣告採用序列模型
model= Sequential()
#卷積層1與池化層1
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))
model.add(Dropout(rate=0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#卷積層2與池化層2
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
#建立神經網路(平坦層、隱藏層、輸出層)
model.add(Flatten())
model.add(Dropout(0.25))
#建立全網路連接層
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
#建立輸出層
model.add(Dense(10, activation='softmax'))
#檢查model 的STACK
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
train_history_bin=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1) 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(train_history.history['accuracy'])
plt.plot(train_history_MSE.history['accuracy'])
plt.plot(train_history_bin.history['accuracy'])
plt.title('Train History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['categorical_crossentropy', 'MSE','binary_crossentropy'], loc='upper left')
plt.show()

<img src="imgs/keras05_loss function_acc.png">
<img src="imgs/keras05_loss function_val_acc.png">